In [23]:
import sys
import pickle
import codecs
sys.path.append('mlp_trainer')

# import GCP Demo 1 package
import trainer.data as data
import trainer.model as model
import tensorflow as tf
from google.cloud import bigquery_storage_v1beta1
tf.compat.v1.enable_eager_execution()

In [7]:
session, readers = data.get_data_partition_sharded("finaltaxi_encoded_sampled", "training", shards=20)
print(session.streams[0].name)
sp = pickle.dumps(session)
print(pickle.loads(sp))
print(readers[0])


In [2]:
dataset = tf.data.Dataset.from_generator(
    model.generator_input,
    (tf.float64, tf.uint16),
    output_shapes=(tf.TensorShape([None]), tf.TensorShape([])),
    args = (            "finaltaxi_encoded_sampled_small",
        1,
        8,
        'train')
).batch(2).repeat(2)

In [6]:
count = 0
for value in dataset.take(1000000):
    if count % 100000 == 0:
        print(value)
    count += 1 

In [29]:
def get_reader_for_stream(session_pickled: bytes, stream_name_bytes: bytes):
    session = pickle.loads(codecs.decode(session_pickled, "base64"))
    stream_name = stream_name_bytes.decode("utf-8")
    client = bigquery_storage_v1beta1.BigQueryStorageClient()
    for stream in session.streams:
        if stream.name == stream_name:
            reader = data.get_reader(client, stream)
            rows = reader.rows(session)
            for row in rows:
                # df = pd.DataFrame([row])
                yield (
                    # df.drop(['cash', ], axis=1).values,
                    # df['cash'].values 
                    [
                        row.get("year"),
                        row.get("start_time_norm_midnight"),
                        row.get("start_time_norm_noon"),
                        row.get("pickup_lat_std"),
                        row.get("pickup_long_std"),
                        row.get("pickup_lat_centered"),
                        row.get("pickup_long_centered"),
                        row.get("day_of_week_MONDAY"),
                        row.get("day_of_week_TUESDAY"),
                        row.get("day_of_week_WEDNESDAY"),
                        row.get("day_of_week_THURSDAY"),
                        row.get("day_of_week_FRIDAY"),
                        row.get("day_of_week_SATURDAY"),
                        row.get("day_of_week_SUNDAY"),
                        row.get("month_JANUARY"),
                        row.get("month_FEBRUARY"),
                        row.get("month_MARCH"),
                        row.get("month_APRIL"),
                        row.get("month_MAY"),
                        row.get("month_JUNE"),
                        row.get("month_JULY"),
                        row.get("month_AUGUST"),
                        row.get("month_SEPTEMBER"),
                        row.get("month_OCTOBER"),
                        row.get("month_NOVEMBER"),
                        row.get("month_DECEMBER"),
                    ],
                    row.get("cash")
                )

def bq_stream_generator(table_id: bytes, partition: bytes):
    session, _ = data.get_data_partition_sharded(table_id.decode("utf-8"), partition.decode("utf-8"), shards=100)
    encoded_session = codecs.encode(pickle.dumps(session), "base64")
    for stream in session.streams:
        yield(encoded_session, stream.name)

dataset = tf.data.Dataset.from_generator(
    bq_stream_generator,
    (tf.string, tf.string),
    # output_shapes=(tf.TensorShape([]), tf.TensorShape([])),
    args = (            "finaltaxi_encoded_sampled",
        'train')
).interleave(lambda session, stream: 
    tf.data.Dataset.from_generator(
        get_reader_for_stream,
        (tf.float64, tf.uint16),
        args=(session, stream)
    ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE),
    num_parallel_calls=tf.data.experimental.AUTOTUNE
).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).batch(2).repeat(2)

count = 0
for value in dataset.take(1):
    # if count % 100000 == 0:
    print(value)
    count += 1 

(<tf.Tensor: id=1247, shape=(2, 26), dtype=float32, numpy=
array([[ 2.0160000e+03,  4.2708829e-01, -1.4583333e-01,  1.7103280e+00,
        -8.0152071e-01, -8.4169000e-02,  8.9659065e-02,  0.0000000e+00,
         1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00],
       [ 2.0130000e+03,  6.4584082e-01,  2.9167342e-01, -3.6133602e-02,
        -1.1567656e-01, -1.5770180e-02,  3.7937798e-02,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  1.0000000e+